In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import csv

import plate_specification

In [ ]:
primer_validate_mapping = plate_specification.well_mapping({
    'hActB-r1': 'A1-E1',
    'hActB-r2': 'A2-E2',
    'hGADPH-r1': 'A3-E3',
    'hGADPH-r2': 'A4-E4',
    'Lenti-5p-1': 'A5-E5',
    'Lenti-5p-2': 'A6-E6',
    'EGFP-1': 'A7-E7',
    'EGFP-2': 'A8-E8',
    'interstitial-1': 'A9-E9',
    'interstitial-2': 'A10-E10',
    'mRuby2-1': 'A11-E11',
    'mRuby2-2': 'A12-E12',
    'Lenti-3p-1': 'F1-F5',
    'Lenti-3p-2': 'G1-G5',
    'BB-1': 'F6-F10',
    'BB-2': 'G6-G10', 
    'none': 'H1-H3',
    'water': 'A1-A12,F1-G1,F6-G6,H1-H3',
    'no-plasmid': 'B1-B12,F2-G2,F7-G7',
    'tandem': 'C1-C12,F3-G3,F8-G8',
    'convergent': 'D1-D12,F4-G4,F9-G9',
    'divergent': 'E1-E12,F5-G5,F10-G10'
})

In [ ]:
def map_qPCR_data(file, mapping, token_names=['primerset', 'sample']):
    tokens = [[] for _ in range(len(token_names))]
    cts = []
    with open(file, 'r') as ct_file:
        # Ignore first line
        next(ct_file)
        ct_csv = csv.DictReader(ct_file, dialect='excel-tab')
        for line in ct_csv:
            if line['Pos'] not in mapping or len(line['Cp']) == 0:
                continue
            full_sample = mapping[line['Pos']]
            tokenized_sample = full_sample.split('.')
            while len(tokenized_sample) < len(token_names):
                tokenized_sample.append('')
            for id, token in enumerate(tokenized_sample):
                tokens[id].append(token)
            cts.append(float(line['Cp']))
    data = {k:v for k,v in zip(token_names, tokens)}
    data.update({'Ct':cts})
    return pd.DataFrame(data=data)

In [ ]:
validation_df = map_qPCR_data('../data/TMP_qPCR/2021-07-16-TMP_primer_validation.txt', primer_validate_mapping)

In [ ]:
plt.figure(figsize=(20,5))
sns.barplot(x='primerset', y='Ct', hue='sample', data=validation_df)
plt.savefig('../img/qPCR_controls.pdf')
plt.show()

In [ ]:
pre_exo_mapping = plate_specification.well_mapping({
    'hActB': 'A1-B12,C1-C9',
    'BB-1': 'D1-E12,F1-F9,G1',
    'water': 'A1,D1,G1',
    'direct_PEI': 'A2,D2',
    'direct_tandem': 'A3-A5,D3-D5',
    'direct_convergent': 'A6,D6',
    'direct_divergent': 'A7,D7',
    '-UV_PEI': 'A8,D8',
    'PEI': 'A9,D9',
    '-UV_tandem': 'A10-A11,D10-D11',
    'tandem': 'A12,B1-B5,D12,E1-E5',
    '-UV_convergent': 'B6-B7,E6-E7',
    'convergent': 'B8-B12,C1,E8-E12,F1',
    '-UV_divergent': 'C2-C3,F2-F3',
    'divergent': 'C4-C9,F4-F9',
    '-dox': 'A10,A12,B1-B2,B6,B8-B10,C2,C4-C6,D10,D12,E1-E2,E6,E8-E10,F2,F4-F6',
    '+dox': 'A11,B3-B5,B7,B11-B12,C1,C3,C7-C9,D11,E3-E5,E7,E11-E12,F1,F3,F7-F9',
    'R1': 'A3,A10-A12,B3,B6-B8,B11,C2-C4,C7,D3,D10-D12,E3,E6-E8,E11,F2-F4,F7',
    'R2': 'A4,B1,B4,B9,B12,C5,C8,D4,E1,E4,E9,E12,F5,F8',
    'R3': 'A5,B2,B5,B10,C1,C6,C9,D5,E2,E5,E10,F1,F6,F9'
})

In [ ]:
exo_norm_df = map_qPCR_data('../data/TMP_qPCR/2021-07-22-pre_exo_normalization.txt', pre_exo_mapping, ['primerset', 'sample', 'dox', 'replicate'])
exo_norm_df['condition'] = exo_norm_df['sample'] + '.' + exo_norm_df['dox'] + '.'
exp_only = exo_norm_df[(exo_norm_df['sample'] != 'water') & (~exo_norm_df['condition'].str.endswith('PEI..'))]
cross_df = exp_only.pivot(index=['condition','replicate'], columns='primerset', values='Ct')

In [ ]:
plt.figure(figsize=(30,5))
sns.scatterplot(x='condition', y='Ct', hue='primerset', data=exo_norm_df)
plt.show()
plt.figure(figsize=(5,5))
sns.scatterplot(x='replicate', y='Ct', hue='primerset', data=exo_norm_df[exo_norm_df['condition']=='tandem.-dox.'])
plt.show()
plt.figure()
sns.scatterplot(x='hActB', y='BB-1', data=cross_df)
plt.xlabel('hActB Ct ($-\log_2(c)$)')
plt.ylabel('Backbone Ct ($-\log_2(c)$)')
plt.savefig('../img/pre_dilute_qPCR_comparison.svg')

In [ ]:
cross_df['dilution_factor'] = np.exp2(cross_df['hActB']- 19.24)
cross_df['uL water'] = 100.0 * (1.0 - cross_df['dilution_factor'])
cross_df

In [ ]:
divergent_test_mapping = plate_specification.well_mapping({
    'hActB': 'B2-B10,E2-E7,H2-H10,K2-K8',
    'BB-1':  'C2-C10,F2-F7,I2-I10,L2-L8',
    'interstitial-1': 'D2-D10,G2-G7,J2-J10,M2-M8',
    'direct-Divergent': 'B2-D4,H2-J4',
    '-UV-Divergent': 'B5-D10,H5-J10',
    'Divergent': 'E2-G7,K2-M7',
    '-Dox': 'B2-D7,E2-G4,H2-J7,K2-M4',
    '+Dox': 'B8-D10,E5-G7,H8-J10,K5-M7',
    'water': 'K8-M8',
    '-exo': 'B2-D10,E2-G7',
    '+exo': 'H2-J10,K2-M7'
})
tan_con_test_mapping = plate_specification.well_mapping({
    'hActB': 'A1-A24,D1-D24,G1-G24,J1-J24,M1-M12',
    'BB-1': 'B1-B24,E1-E24,H1-H24,K1-K24,N1-N12',
    'interstitial-1': 'C1-C24,F1-F24,I1-I24,L1-L24,O1-O12',
    'direct-Tandem..': 'A1-C3,G7-I9',
    '-UV-Tandem.-Dox.': 'A4-C6,G10-I12',
    '-UV-Tandem.+Dox.': 'A7-C9,G13-I15',
    'direct-Convergent..': 'A10-C12,G16-I18',
    '-UV-Convergent.-Dox.': 'A13-C15,G19-I21',
    '-UV-Convergent.+Dox.': 'A16-C18,G22-I24',
    'Tandem.-Dox.1': 'A19-C21,J1-L3',
    'Tandem.-Dox.2': 'A22-C24,J4-L6',
    'Tandem.-Dox.3': 'D1-F3,J7-L9',
    'Tandem.+Dox.1': 'D4-F6,J10-L12',
    'Tandem.+Dox.2': 'D7-F9,J13-L15',
    'Tandem.+Dox.3': 'D10-F12,J16-L18',
    'Convergent.-Dox.1': 'D13-F15,J19-L21',
    'Convergent.-Dox.2': 'D16-F18,J22-L24',
    'Convergent.-Dox.3': 'D19-F21,M1-O3',
    'Convergent.+Dox.1': 'D22-F24,M4-O6',
    'Convergent.+Dox.2': 'G1-I3,M7-O9',
    'Convergent.+Dox.3': 'G4-I6,M10-O12',
    '-exo': 'A1-F24,G1-I6',
    '+exo': 'G7-L24,J1-O12'
})

In [ ]:
div_test_data = map_qPCR_data('../data/TMP_qPCR/2021.07.16-Divergent-TMP.txt', divergent_test_mapping, ['primerset', 'sample', 'dox', 'exo'])
div_test_data['full_condition'] = div_test_data['sample'] + '.' + div_test_data['dox'] + '.' + div_test_data['exo']
tan_con_test_data = map_qPCR_data('../data/TMP_qPCR/2021.08.11-Tandem_Convergent.txt', tan_con_test_mapping, ['primerset', 'sample', 'dox', 'rep', 'exo'])
tan_con_test_data['full_condition'] = tan_con_test_data['sample'] + '.' + tan_con_test_data['dox'] + '.' + tan_con_test_data['rep'] + tan_con_test_data['exo']

In [ ]:
plt.figure(figsize=(35,5))
sns.barplot(x='full_condition', y='Ct', hue='primerset', data=div_test_data)
plt.show()
plt.figure(figsize=(120,5))
sns.barplot(x='full_condition', y='Ct', hue='primerset', data=tan_con_test_data)
plt.show()

In [ ]:
normed_div_data = div_test_data.copy().groupby(['sample','dox','exo','primerset']).median()
average_exo_reduction = normed_div_data.xs('+exo', level='exo')['Ct'] - normed_div_data.xs('-exo', level='exo')['Ct']
# Use the -exo condition with the backbone primer to normalize for plasmid concentration. Adjust all Ct's accordingly.
normed_div_data['normed_Ct'] = normed_div_data['Ct'] - normed_div_data.xs('-exo', level='exo').xs('BB-1', level='primerset')['Ct']
dox_changes = normed_div_data.xs('+Dox', level='dox')['normed_Ct'] - normed_div_data.xs('-Dox', level='dox')['normed_Ct']
normed_div_data = normed_div_data.reset_index()
normed_div_data['full_condition'] = normed_div_data['sample'] + '.' + normed_div_data['dox'] + '.' + normed_div_data['exo']
print(f'Average exo fold reduction: {2**average_exo_reduction.mean():.2f}')

In [ ]:
dox_changes

In [ ]:
plt.figure(figsize=(35,5))
sns.barplot(x='full_condition', y='normed_Ct', hue='primerset', data=normed_div_data.dropna())
plt.savefig('../img/all_normed_primers.pdf')
plt.show() 

In [ ]:
plt.figure(figsize=(35,5))
sns.factorplot(x='sample', y='normed_Ct', hue='dox', col='exo', data=normed_div_data[normed_div_data['primerset']=='interstitial-1'].dropna(), kind='bar')
#sns.barplot(x=['sample','exo'], y='normed_Ct', hue='dox', data=normed_div_data[normed_div_data['primerset']=='interstitial-1'])
plt.savefig('../img/interstitial_output.pdf')
plt.show() 

In [ ]:
normed_orientation['rep'].map(str)

In [ ]:
orientation_comparison

In [ ]:
orientation_comparison = pd.concat([div_test_data,tan_con_test_data])
orientation_comparison.loc[orientation_comparison['rep'].isna(), 'rep'] = 1
orientation_comparison = orientation_comparison.copy().groupby(['sample', 'dox', 'rep', 'exo', 'primerset']).median()
normed_orientation = orientation_comparison['Ct'] - orientation_comparison.xs('BB-1', level='primerset')['Ct']
normed_orientation = normed_orientation.reset_index()
normed_orientation = normed_orientation[normed_orientation['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
orientation_comparison = orientation_comparison.reset_index()
orientation_comparison['samplerep'] = orientation_comparison['sample'] + '-' + orientation_comparison['rep'].map(str)
normed_orientation['samplerep'] = normed_orientation['sample'] + '-' + normed_orientation['rep'].map(str)
orientation_comparison = orientation_comparison[orientation_comparison['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
grid = sns.catplot(x='samplerep', y='Ct', hue='dox', hue_order=['-Dox','+Dox'], row='primerset', col='exo', height=7, aspect=1,
                   data=normed_orientation[(
                       normed_orientation['primerset']=='interstitial-1') &
                       (normed_orientation['exo']=='+exo')], kind='bar')
for axis in grid.axes.flat:
    axis.set_ylabel('$\log_2$(positive supercoiling)')
plt.savefig('../img/direct_normed.svg')

In [ ]:
orientation_comparison = pd.concat([div_test_data,tan_con_test_data])
orientation_comparison.loc[orientation_comparison['rep'].isna(), 'rep'] = 1
orientation_comparison = orientation_comparison.copy().groupby(['sample', 'dox', 'rep', 'exo', 'primerset']).median()
normed_orientation = orientation_comparison['Ct'] - orientation_comparison.xs('+exo', level='exo').xs('hActB', level='primerset')['Ct']
normed_orientation = normed_orientation.reset_index()
normed_orientation = normed_orientation[normed_orientation['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
orientation_comparison = orientation_comparison.reset_index()
orientation_comparison['samplerep'] = orientation_comparison['sample'] + '-' + orientation_comparison['rep'].map(str)
normed_orientation['samplerep'] = normed_orientation['sample'] + '-' + normed_orientation['rep'].map(str)
orientation_comparison = orientation_comparison[orientation_comparison['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
grid = sns.catplot(x='samplerep', y='Ct', hue='dox', hue_order=['-Dox','+Dox'], row='primerset', col='exo', height=7, aspect=1,
                   data=normed_orientation[(
                       normed_orientation['primerset']=='interstitial-1') &
                       (normed_orientation['exo']=='+exo')], kind='bar')
for axis in grid.axes.flat:
    axis.set_ylabel('$\log_2$(positive supercoiling)')
plt.savefig('../img/actB_normed.svg')
plt.show()

In [ ]:
orientation_comparison = pd.concat([div_test_data,tan_con_test_data])
orientation_comparison.loc[orientation_comparison['rep'].isna(), 'rep'] = 1
orientation_comparison = orientation_comparison.copy().groupby(['sample', 'dox', 'rep', 'exo', 'primerset']).median()
normed_orientation = orientation_comparison['Ct'] - orientation_comparison.xs('-exo', level='exo').xs('hActB', level='primerset')['Ct']
normed_orientation = normed_orientation.reset_index()
normed_orientation = normed_orientation[normed_orientation['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
orientation_comparison = orientation_comparison.reset_index()
orientation_comparison['samplerep'] = orientation_comparison['sample'] + '-' + orientation_comparison['rep'].map(str)
normed_orientation['samplerep'] = normed_orientation['sample'] + '-' + normed_orientation['rep'].map(str)
orientation_comparison = orientation_comparison[orientation_comparison['sample'].map(lambda x: x[0] in ['T', 'C', 'D'])]
grid = sns.catplot(x='samplerep', y='Ct', hue='dox', hue_order=['-Dox','+Dox'], row='primerset', col='exo', height=7, aspect=1,
                   data=normed_orientation[(
                       normed_orientation['primerset']=='interstitial-1') &
                       (normed_orientation['exo']=='+exo')], kind='bar')
for axis in grid.axes.flat:
    axis.set_ylabel('$\log_2$(positive supercoiling)')
plt.savefig('../img/-exo_normed.svg')
plt.show()

In [ ]:
orientation_comparison

In [ ]:

#orientation_comparison = orientation_comparison.copy().groupby(['sample','dox','exo','primerset','rep']).median()
"""
normed_orientation_compar
normed_div_data = div_test_data.copy().groupby(['sample','dox','exo','primerset']).median()
average_exo_reduction = normed_div_data.xs('+exo', level='exo')['Ct'] - normed_div_data.xs('-exo', level='exo')['Ct']
# Use the -exo condition with the backbone primer to normalize for plasmid concentration. Adjust all Ct's accordingly.
normed_div_data['normed_Ct'] = normed_div_data['Ct'] - normed_div_data.xs('-exo', level='exo').xs('BB-1', level='primerset')['Ct']
dox_changes = normed_div_data.xs('+Dox', level='dox')['normed_Ct'] - normed_div_data.xs('-Dox', level='dox')['normed_Ct']
normed_div_data = normed_div_data.reset_index()
normed_div_data['full_condition'] = normed_div_data['sample'] + '.' + normed_div_data['dox'] + '.' + normed_div_data['exo']
print(f'Average exo fold reduction: {2**average_exo_reduction.mean():.2f}')
normed_div_data = div_test_data.copy().groupby(['sample','dox','exo','primerset']).median()
"""